In [45]:
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors,Vector
from pyspark.mllib.classification import SVMModel, SVMWithSGD
from pyspark.python.pyspark.shell import spark
from pyspark.sql.types import Row
from pyspark.sql.types import *
import pandas as pd
from pyspark import SparkConf
from pyspark.sql import SparkSession
from operator import add
from pyspark.sql import HiveContext
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [2]:
spark = SparkSession.builder \
    .appName('spark_project') \
    .config('spark.executor.memory','2g') \
    .enableHiveSupport() \
    .getOrCreate()
sc = spark.sparkContext


In [56]:
train_data = sc.textFile('/Users/chenyuanshan/temp/data/data/featureMatrix/trainfill.csv')
# test_data = sc.testFile('/Users/chenyuanshan/temp/data/data/testMatrix/testfill.csv')

In [57]:
def GetParts(line):
    parts = line.split(',')
    return LabeledPoint(float(parts[11]),Vectors.dense(float(parts[1]),float(parts[2]),float(parts[3]),float(parts[4]), \
                                                      float(parts[5]), float(parts[6]), float(parts[7]), float(parts[8]), \
                                                      float(parts[9]), float(parts[10])))
train = train_data.map(lambda line: GetParts(line))

In [42]:
test_data = spark.read.format('com.databricks.spark.csv').options(header='false', inferschema='true').load('/Users/chenyuanshan/temp/data/data/testMatrix/testfill.csv')
test_data_1 = test_data.rdd

In [43]:
# test = test_data_1.map(lambda parts : LabeledPoint(parts[11],Vectors.dense(float(parts[1]),float(parts[2]),float(parts[3]),float(parts[4]), \
#                                                      float(parts[5]), float(parts[6]), float(parts[7]), float(parts[8]), \
#                                                      float(parts[9]), float(parts[10]))))

In [58]:
ttest = test_data_1.map(lambda parts : LabeledPoint(1.0,Vectors.dense(parts[1],parts[2],parts[3],parts[4], \
                                                      parts[5], parts[6], parts[7], parts[8], \
                                                      parts[9], parts[10])))

In [59]:
train.take(5)

[LabeledPoint(0.0, [4718.0,6.0,0.0,62.0,3.0,0.0,1.0,0.0,0.3333333333333333,0.08940820834786653]),
 LabeledPoint(0.0, [3152.0,3.0,0.0,591.0,16.0,0.0,1.0,0.0,0.0625,0.09346991037131884]),
 LabeledPoint(0.0, [4377.0,6.0,1.0,537.0,3.0,0.0,1.0,0.0,0.3333333333333333,0.12137006701414745]),
 LabeledPoint(0.0, [2760.0,0.0,0.0,49.0,3.0,0.0,2.0,0.0,0.6666666666666666,0.07458405048766495]),
 LabeledPoint(0.0, [3885.0,4.0,1.0,182.0,0.0,0.0,1.0,0.0,0.6666666666666666,0.12729948491537896])]

In [60]:
ttest.take(5)

[LabeledPoint(1.0, [1970.0,5.0,0.0,54.0,5.0,0.0,1.0,0.0,0.2,0.19913419913419916]),
 LabeledPoint(1.0, [3936.0,2.0,0.0,18.0,4.0,0.0,1.0,1.0,0.25,0.09519430866766684]),
 LabeledPoint(1.0, [4044.0,3.0,0.0,324.0,21.0,0.0,2.0,1.0,0.09523809523809523,0.027913672813971814]),
 LabeledPoint(1.0, [1557.0,7.0,1.0,43.0,2.0,0.0,1.0,1.0,0.5,0.07266811279826464]),
 LabeledPoint(1.0, [2991.0,0.0,0.0,37.0,2.0,0.0,1.0,0.0,0.5,0.17016609213412726])]

In [61]:
numIterations = 1000
model = SVMWithSGD.train(train, numIterations)

In [62]:
labelsAndPreds = train.map(lambda p: (p.label, model.predict(p.features)))

In [63]:
testOutput = ttest.map(lambda x : model.predict(x.features))

In [64]:
svmTotalCorrect = train.map(lambda x: 1 if model.predict(x.features) == x.label else 0).sum()

In [65]:
svmAccuracy = float(svmTotalCorrect)/train.count()
print(svmAccuracy)

0.9388378618743867


In [53]:
testOutput.take(10)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

想获得probability，直接获得label了怎么办？

In [66]:
sum(testOutput.collect())

17

In [20]:
len(testOutput.collect())
len(train.collect())

260864

In [67]:
scoreAndLabels = train.map(lambda x:(float(model.predict(x.features)),x.label))
metrics = BinaryClassificationMetrics(scoreAndLabels)
print('PR:{:.4f}, AUC:{:.4f}'.format(metrics.areaUnderPR, metrics.areaUnderROC))

PR:0.2229, AUC:0.5001


The outcome is not satisfired enough. Try RandomForestClassifier.

In [32]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer

In [26]:
rf_trainData = spark.read.csv('/Users/chenyuanshan/temp/data/data/featureMatrix/trainfill.csv',header=False)

In [28]:
rf_dataSet = rf_trainData.rdd.map(list)
rf_trainData, rf_testData= rf_dataSet.randomSplit([0.7, 0.3], seed=7)

In [30]:
rf_trainingSet = rf_trainData.map(lambda x:Row(label=x[-1], features=Vectors.dense(x[2:10]))).toDF()
rf_train_num = rf_trainingSet.count()
print(rf_train_num)

182682


In [37]:
stringIndexer = StringIndexer(inputCol="label", outputCol="indexed")
si_model = stringIndexer.fit(rf_trainingSet)
train_tf = si_model.transform(rf_trainingSet)
train_tf.show(5)
rf_classifier=RandomForestClassifier(labelCol="indexed",numTrees=50).fit(train_tf)
# rf = RandomForestClassifier(numTrees=100, labelCol="indexed", seed=7)
rfModel = rf.fit(train_tf)

+-----+--------------------+-------+
|label|            features|indexed|
+-----+--------------------+-------+
|    0|[6.0,0.0,62.0,3.0...|    0.0|
|    0|[3.0,0.0,591.0,16...|    0.0|
|    0|[6.0,1.0,537.0,3....|    0.0|
|    0|[0.0,0.0,49.0,3.0...|    0.0|
|    0|[4.0,1.0,182.0,0....|    0.0|
+-----+--------------------+-------+
only showing top 5 rows



IllegalArgumentException: requirement failed: Column features must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually struct<type:tinyint,size:int,indices:array<int>,values:array<double>>.

In [68]:
trainSet_rf = pd.read_csv('/Users/chenyuanshan/temp/data/data/featureMatrix/withHeader.csv')

In [69]:
trainSet_rf.info

<bound method DataFrame.info of         user_id  merchant_id  age_range  gender  total_log  click  add_chart  \
0           464         4718        6.0     0.0         62      3          0   
1           867         3152        3.0     0.0        591     16          0   
2          1882         4377        6.0     1.0        537      3          0   
3          2450         2760        0.0     0.0         49      3          0   
4          2766         3885        4.0     1.0        182      0          0   
...         ...          ...        ...     ...        ...    ...        ...   
260859   421807         3057        3.0     1.0         34      6          0   
260860   422078         3374        4.0     0.0        101      1          0   
260861   422097         3609        3.0     0.0        275      0          0   
260862   422648         4502        3.0     0.0         81      3          0   
260863   423267         3578        4.0     1.0         46      3          0   

       

In [70]:
from pyspark.ml.feature import VectorAssembler

In [71]:
dfInfoAssembler = VectorAssembler(inputCols=['age_range','gender','total_log','click','add_chart','buy','favourite', \
                                            'store_buy_rate', 'store_rate'],outputCol='features')

In [72]:
dfInfo = dfInfoAssembler.transform(trainSet_rf)

AttributeError: 'DataFrame' object has no attribute '_jdf'